In [53]:
import casparser
import json
from jsonpath_ng import parse

def formatFolioNumInCas(folioNumInCas):
    if '/ 0' in folioNumInCas:
        # Extract the part before '/ 0'
        extracted_data = folioNumInCas.split(' / ')[0]
        return extracted_data
    else:
        # Remove spaces and format the data
        return folioNumInCas.replace(' ', '')

def restructure_data(jsonObj):
    result_dict = {}
    folios = jsonObj.get('folios', [])

    for folio_data in folios:
        pan = folio_data.get('PAN')
        folio = folio_data.get('folio')
        schemes = folio_data.get('schemes', [])
        
        if pan not in result_dict:
            result_dict[pan] = []
        
        filtered_schemes = [{"scheme": scheme.get("scheme"), "valuation": scheme.get("valuation")} for scheme in schemes]
        
        result_dict[pan].append({"folio": folio, "schemes": filtered_schemes})
    
    return result_dict

def updateCasJsonByPan(casJsonByPan, casJson):
    for folioData in casJson.get('folios', []):
        pan = folioData.get('PAN')
        folio = formatFolioNumInCas(folioData.get('folio'))
        schemes = folioData.get('schemes', [])
        filtered_schemes = [{"scheme": scheme.get("scheme"), "valuation": scheme.get("valuation")} for scheme in schemes]
        if pan not in casJsonByPan:
            casJsonByPan[pan] = {}
        if folio not in casJsonByPan[pan]:
            casJsonByPan[pan][folio] = {'schemes': schemes}
        else:
            casJsonByPan[pan].get(folio).get('schemes').append(schemes)

In [51]:
casJson0 = json.loads(casparser.read_cas_pdf("/Users/akshay.m/Downloads/32109276320221148Q417371871VC77CPIMBCP158179714.pdf", "Akshay@1997", output="json"))
casJson1 = json.loads(casparser.read_cas_pdf("/Users/akshay.m/Downloads/71218376320221145S8BDE019977851J351CPIMBCP158179910.pdf", "Akshay@1997", output="json"))

In [54]:
casJsonByPan = {}
updateCasJsonByPan(casJsonByPan, casJson0)
updateCasJsonByPan(casJsonByPan, casJson1)
print(json.dumps(casJsonByPan))

{"ADYPM3720C": {"910160728593": {"schemes": [{"scheme": "Axis Growth Opportunities Fund Direct Growth", "advisor": "INZ000208032", "rta_code": "128OGDGG", "rta": "KFINTECH", "type": "EQUITY", "isin": "INF846K01J46", "amfi": "145110", "nominees": ["AKSHAY MITTAL"], "open": "0.000", "close": "950.523", "close_calculated": "950.523", "valuation": {"date": "2023-12-20", "nav": "26.85", "cost": "20000.00", "value": "25521.54"}, "transactions": [{"date": "2022-09-26", "description": "Purchase", "amount": "19999.00", "units": "950.523", "nav": "21.04", "balance": "950.523", "type": "PURCHASE", "dividend_rate": null}, {"date": "2022-09-26", "description": "*** Stamp Duty ***", "amount": "1.00", "units": null, "nav": null, "balance": null, "type": "STAMP_DUTY_TAX", "dividend_rate": null}]}]}, "26513979": {"schemes": [{"scheme": "ICICI Prudential Nifty 50 Index Fund - Direct Plan Growth (Non-Demat", "advisor": "INZ000208032", "rta_code": "P8189", "rta": "CAMS", "type": "EQUITY", "isin": "INF109K

In [32]:
import csv

# Initialize an empty list to store the dictionaries
currentDataList = []

# Open the CSV file and read its content into a list of dictionaries
with open('/Users/akshay.m/Downloads/Portfolio/Mutual-funds-Mutual-fund-investments.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        currentDataList.append(dict(row))

# Print the data list containing dictionaries
print(json.dumps(currentDataList))


[{"Symbol": "0P00005X02.BO", "Fund name": "ICICI Prudential Multi-Asset Fu", "Common name": "", "Owner": "Charu Mittal", "Folio No.": "17694516/90", "Transaction Type": "Buy", "Mode of Investment": "Regular", "Date of investment": "23 Oct 2023", "Date of redemption": "", "Dated NAV (Default to 10)": "\u20b9547.63", "Invested amount": "\u20b9136,080.01", "Units": "248.49", "Recent/Redeem NAV": "\u20b9584.69", "Present/Redeemed market value": "\u20b9145,286.73", "Days passed since/Held Duration": "60", "Remaining number of days for maturity": "306", "Net Gain": "\u20b99,206.72", "Absolute gain": "6.77%", "CAGR": "48.92%", "Maturity Date": "23 Oct 2024", "Time to reach LTCG taxation in days": "366", "Label": "", "Transaction Status": "Completed", "Notes": "", "": "", "Symbol filter": "TRUE", "Owner filter": "TRUE", "Folio No. Filter": "TRUE", "Transaction type Filter": "TRUE", "Mode of investment filler": "TRUE", "Maturity filter": "TRUE", "Time to reach LTCG taxation filter": "TRUE", "Ab

In [49]:
currentDataByPan = {}
ownerSet = set()
for row in currentDataList:
    transactionType = row.get('Transaction Type')
    if (transactionType != 'Buy'):
        continue
    owner = row.get('Owner')
    if (owner == 'Charu Mittal'):
        pan = 'ADYPM3720C'
    elif (owner == 'Akshay Mittal'):
        pan = 'DDMPM6612L'
    elif (owner == 'Ramesh Chand'):
        pan = 'ABUPC0136G'
    elif (owner == 'Sanjay Kumar Mittal'):
        pan = 'AAXPK9504E'
    else:
        continue
    if pan not in currentDataByPan:
        currentDataByPan[pan] = {}
    currentDataByPanPan = currentDataByPan[pan]
    rowFolioNum = row.get('Folio No.')
    if rowFolioNum not in currentDataByPanPan:
        currentDataByPanPan[rowFolioNum] = {'schemes': []}
    currentScheme = {}
    currentDataByPanPan[rowFolioNum].get('schemes').append(currentScheme)
    currentScheme['yahooSymbol'] = row.get('Symbol')
    currentSchemeValuation = {}
    currentScheme['valuation'] = currentSchemeValuation
    currentSchemeValuation['cost'] = row.get('Invested amount')
    currentSchemeValuation['value'] = row.get('Present/Redeemed market value')
print(json.dumps(currentDataByPan))

{"ADYPM3720C": {"17694516/90": {"schemes": [{"yahooSymbol": "0P00005X02.BO", "valuation": {"cost": "\u20b9136,080.01", "value": "\u20b9145,286.73"}}, {"yahooSymbol": "0P00008TN4.BO", "valuation": {"cost": "\u20b9149,992.50", "value": "\u20b9174,080.56"}}, {"yahooSymbol": "0P0001LF3S.BO", "valuation": {"cost": "\u20b9139,993.00", "value": "\u20b9184,325.83"}}, {"yahooSymbol": "0P00005UNZ.BO", "valuation": {"cost": "\u20b9149,992.50", "value": "\u20b9174,769.88"}}, {"yahooSymbol": "0P0001HO3I.BO", "valuation": {"cost": "\u20b959,997.00", "value": "\u20b974,899.69"}}, {"yahooSymbol": "0P0001HO3I.BO", "valuation": {"cost": "\u20b940,000.00", "value": "\u20b947,538.30"}}, {"yahooSymbol": "0P00005UNZ.BO", "valuation": {"cost": "\u20b9149,993.00", "value": "\u20b9187,636.61"}}, {"yahooSymbol": "0P00005UNZ.BO", "valuation": {"cost": "\u20b9101,995.00", "value": "\u20b9136,081.88"}}, {"yahooSymbol": "0P00008TN4.BO", "valuation": {"cost": "\u20b934,998.00", "value": "\u20b946,195.84"}}]}, "27784

In [23]:
import yfinance
ticker = yfinance.Ticker('0P0000XV6P.BO')

In [25]:
ticker.info

{'address1': 'G Block, BKC, Bandra (East)',
 'address2': '2nd Floor, 12-BKC, Plot No C-12',
 'address3': 'Kotak Mahindra Asset management Company Limited',
 'phone': '+91 22 61152100',
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 292.605,
 'regularMarketPreviousClose': 292.605,
 'trailingPE': 30.730618,
 'averageVolume': 0,
 'averageVolume10days': 0,
 'averageDailyVolume10Day': 0,
 'yield': 0.0,
 'ytdReturn': 0.1552741,
 'totalAssets': 6844143104,
 'fiftyTwoWeekLow': 222.527,
 'fiftyTwoWeekHigh': 299.259,
 'fiftyDayAverage': 281.39252,
 'twoHundredDayAverage': 258.78918,
 'currency': 'INR',
 'morningStarOverallRating': 4,
 'morningStarRiskRating': 2,
 'annualReportExpenseRatio': 0.0,
 'beta3Year': 0.84,
 'fundInceptionDate': 1356998400,
 'annualHoldingsTurnover': 0.1595,
 'exchange': 'MCE',
 'quoteType': 'MUTUALFUND',
 'symbol': '0P0000XV6P.BO',
 'underlyingSymbol': '0P0000XV6P.BO',
 'shortName': 'Kotak Equity Opportunities Dire',
 'longName': 'Kotak Equity Opportunities Dir Gr

In [69]:
# result = yfinance.Ticker('INF846K01J46')
# print(result)
result = yfinance.Ticker('INF174KA1EZ1')

In [70]:
try: 
    result.info
except Err

NameError: name 'HTTPError' is not defined